# Navigator Text-SQL Notebook

In [20]:
%%capture
!pip install git+https://github.com/gretelai/gretel-python-client@dev/data-designer-m1

session_kwargs = {
    "api_key": "prompt",
    "endpoint": "https://api-dev.gretel.cloud",
    "cache": "yes",
}

In [21]:
from gretel_client.navigator import DataDesigner

### 📘 Text-to-SQL Blueprint
The blueprint below is inspired by our [synthetic text-to-SQL dataset](https://huggingface.co/datasets/gretelai/synthetic_text_to_sql).  

Data Designer can support SQL dialects like ANSI, T-SQL, BigQuery, MySQL, and PostgreSQL. This example uses ANSI

In [22]:
text2sql_blueprint_string ="""
model_suite: Apache-2.0

special_system_instructions: >-
  You are an expert at writing SQL queries and technical documentation. You are obsessed with writing
  clean, efficient, and maintainable SQL code. You are tasked with generating SQL queries and natural
  language text that will be used to train a language model that will be used to generate SQL queries.

seed_categories:
  - name: domain
    description: Major industry domain or sector that relies on robust data solutions
    values: [Healthcare, Finance, Retail, Manufacturing, Education, Public Health, Science and Technology, Environmental Science, Government, Media and Entertainment, 
            Transportation, Energy, Agriculture, Manufacturing, Food and Beverage, Wellness, Construction, Automotive, Telecommunications, Public Services, Financial Services, 
            Medicine, Social Services, Education and Training, Information and Communications, Environment, Textiles, Startups, Legal and Law, Entertainment, Pharmaceuticals, 
            Food Service, Advertising, Financial Planning, Travel and Tourism, Waste Management, E-commerce, Hospitality, Philanthropy, Public Health, Sports, Social Media, 
            Venture Capital, Arts and Culture, Economics, Artificial Intelligence, Biotechnology, Renewable Energy and Sustainability, Business and Entrepreneurship, 
            Defense and Aerospace, Logistics, Oil and Gas, Fashion and Apparel, Human Resources, Music, Nonprofit, Gaming, Insurance, Space Exploration, Banking, Smart Cities, 
            Recreation, Maritime, Electricity, Gas & Water Services, Wholesale Trade, Hotel and Resorts, Rental Services, Fitness, Agricultural Technology, Consulting, Analytics, 
            Chemicals, Urban Planning, Internet of Things, Global Trade, Automation Technology, Journalism, Engineering, Psychology, Scientific Research, Publishing, Cybersecurity, 
            Credit Cards & Loans, Robotics & Computing, Digital Health, Consumer Electronics, Business Intelligence, Market Research, Sales Forecasting, Data Governance, Digital Marketing]
    subcategories:
      - name: topic
        description: Key topics that professional SQL developers care about in the given domain
        num_values_to_generate: 25

  - name: sql_complexity
    description: Complexity of the SQL query, ranging from basic operations to advanced data processing techniques
    values:
      - "Basic SQL"
      - "Aggregation"
      - "Single Join"
      - "Subquery"
      - "Multiple Join"
      - "Window Functions"

  - name: sql_task_type
    description: Type of SQL task that the query represents
    num_values_to_generate: 25
    values:
      - "Data Retrieval"
      - "Data Definition"
      - "Data Manipulation"
      - "Analytics and Reporting"
      - "Database Administration"
      - "Data Cleaning and Transformation"

  - name: natural_language_type
    description: Type of natural language that will be paired with an SQL query
    num_values_to_generate: 25
    values:
      - "a natural language prompt for an SQL query"
      - "a question about how to extract information from a database using SQL"
      - "an explanation of what an SQL query is doing"
      - "an instruction that tells a user to write an SQL query for a specific task"

  - name: data_generation_mode
    description: Mode of data generation for different contexts
    values:
      - "Creation"
      - "Editing"
      - "Augmentation"
      - "Sequential"
      - "Time Series"

data_columns:
  - name: domain_description
    description: Detailed description of the {domain}
    specific_instructions: "Provide a detailed description of the {domain} that includes one key areas of focus for a subdomain for writing SQL."

  - name: sql_complexity_description
    description: Description of the complexity level of the SQL query
    specific_instructions: "Provide a description for the {sql_complexity} level, highlighting the types of SQL operations involved."
    relevant_columns: [sql_complexity]

  - name: sql_task_type_description
    description: Description of the type of SQL task
    specific_instructions: "Provide a description for the {sql_task_type}, including its typical use cases in the {domain} industry."
    relevant_columns: [sql_task_type, domain]

  - name: sql_prompt
    description: Natural language prompt that will be paired with an SQL query
    specific_instructions: "You are an expert engineer, well versed in prompt tuning for LLMs.
    Create a natural language prompt to generate SQL in the field of {domain}, specifically about the domain of {topic}. 
    Feel free to ask for data that focus on a smaller subject within the scope of {natural_language_type}.

    Make sure to use the following guidelines:
        * Just return the generated text without any additional preface or content
        * The generated text must include diverse words / sentences and must always specify all required column names
    "
    relevant_columns: [domain, topic, natural_language_type]

  - name: sql_context
    description: SQL context that provides additional information, such as table or view creation statements
    specific_instructions: "Generate SQL statement as context that is relevant to the {domain} industry and aligns with the {topic} and {sql_prompt}"
    relevant_columns: [domain, topic, sql_prompt]
    output_type: code
    llm_type: code

  - name: sql
    description: SQL query that will be paired with natural language text
    specific_instructions: "Write an SQL query to accomplish the task described by {sql_prompt}. Use the provided {sql_context} if applicable."
    relevant_columns: [domain, topic, sql_complexity, sql_task_type, sql_context, sql_prompt]
    output_type: code
    llm_type: code

  - name: sql_explanation
    description: Natural language explanation of what the SQL query is doing
    specific_instructions: "Provide a detailed explanation of the SQL query, including the purpose of each clause and how it contributes to achieving the task described by {sql_prompt}."
    relevant_columns: [sql_prompt, sql, sql_context]

data_validators:
  - validator: code
    code_lang: ansi
    code_columns: [sql]
"""

In [23]:
# Defines a new DataDesigner instance
designer = DataDesigner.from_config(text2sql_blueprint_string, **session_kwargs)
designer


Found cached Gretel credentials
Using endpoint https://api-dev.gretel.cloud
Logged in as dhruv@gretel.ai ✅


DataDesigner(
    seed_categories (needs generation): ['domain:topic', 'sql_complexity', 'sql_task_type', 'natural_language_type', 'data_generation_mode']
    data_columns: ['domain_description', 'sql_complexity_description', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation']
    validators: ['code']
)

### 👀 Generating a dataset preview

In [24]:
preview = designer.generate_dataset_preview()

[20:56:47] [INFO] 🚀 Generating dataset preview
[20:56:48] [INFO] 🦜 Step 1: Generate seed category values
[21:00:13] [INFO] 🌱 Step 2: Sample data seeds
[21:00:13] [INFO] 🦜 Step 3: Generate column from template >> generating domain description
[21:00:16] [INFO] 🦜 Step 4: Generate column from template >> generating sql complexity description
[21:00:19] [INFO] 🦜 Step 5: Generate column from template >> generating sql task type description
[21:00:22] [INFO] 🦜 Step 6: Generate column from template >> generating sql prompt
[21:00:24] [INFO] 🦜 Step 7: Generate column from template >> generating sql context
[21:00:28] [INFO] 🦜 Step 8: Generate column from template >> generating sql
[21:00:31] [INFO] 🦜 Step 9: Generate column from template >> generating sql explanation
[21:00:36] [INFO] 🔍 Step 10: Validate code
[21:00:38] [INFO] 👀 Your dataset preview is ready for a peek!


In [25]:
preview.display_dataframe_in_notebook()

,sql_complexity,data_generation_mode,sql_task_type,natural_language_type,domain,topic,domain_description,sql_complexity_description,sql_task_type_description,sql_prompt,sql_context,sql,sql_explanation,sql_is_valid,sql_messages
0,Basic SQL,Editing,Role Management,a query to insert new data,Consulting,Predictive Analytics Applications,"The domain of Consulting focuses on providing expert advice and strategic guidance to clients to improve their business operations and performance. A key area of focus is Role Management, which involves defining, creating, and maintaining user roles within a database system to ensure proper access control and data security. This subdomain requires writing SQL queries to manage user roles, such as inserting new roles, updating role permissions, and deleting roles as needed.","The `sql_complexity_description` for Basic SQL level is: Involves simple operations such as SELECT, INSERT, UPDATE, and DELETE on single tables without complex conditions, joins, or subqueries.","The `sql_task_type_description` for Role Management is: Managing user roles and permissions within a database to ensure secure access and compliance with organizational policies. Typical use cases in the Consulting industry include setting up role-based access control for clients, ensuring data privacy, and facilitating secure data sharing among team members.","Please provide the necessary columns (customer_id, purchase_date, product_category, quantity, price) and any additional filters or conditions you would like to apply, so I can generate the SQL query to insert new data into the Predictive Analytics Applications domain.","CREATE TABLE Predictive_Analytics ( customer_id INT, purchase_date DATE, product_category VARCHAR(100), quantity INT, price DECIMAL(10, 2) ); INSERT INTO Predictive_Analytics (customer_id, purchase_date, product_category, quantity, price) SELECT customer_id, purchase_date, product_category, quantity, price FROM Sales_Data WHERE product_category IN ('Consulting Services', 'IT Solutions', 'Data Analytics') AND purchase_date >= DATEADD(year, -1, GETDATE());","INSERT INTO Predictive_Analytics (customer_id, purchase_date, product_category, quantity, price) SELECT customer_id, purchase_date, product_category, quantity, price FROM Sales_Data WHERE product_category IN ('Consulting Services', 'IT Solutions', 'Data Analytics') AND purchase_date >= DATEADD(year, -1, GETDATE());","The SQL query inserts new data into the Predictive_Analytics table from the Sales_Data table. It selects records where the product_category is either 'Consulting Services', 'IT Solutions', or 'Data Analytics' and the purchase_date is within the last year. The query ensures that only relevant data is inserted by applying filters on the product_category and purchase_date.",True,[]
1,Subquery,Creation,Alter Table,a request to recover lost data,Engineering,Data Quality Assurance,"The Engineering domain focuses on the design, development, and maintenance of complex systems. A key area of focus is on writing SQL to manage and manipulate database structures. This includes using subqueries and altering tables to recover lost data, ensuring data integrity and consistency. This task requires a deep understanding of relational database management systems and the ability to craft efficient, error-free SQL commands.","The Subquery level involves SQL queries that incorporate one or more subqueries within the main query. These subqueries can be used in various clauses such as SELECT, FROM, WHERE, and JOIN. Subqueries at this level can perform operations like filtering, aggregation, and joining data from multiple tables, increasing the complexity of the query.","The `sql_task_type_description` for Alter Table is a SQL command used to modify the structure of an existing table. Common use cases in the Engineering industry include adding or removing columns, altering column data types, adding or dropping constraints, and modifying table properties to better fit the 

### 🔎 Taking a closer look at single records

In [26]:
designer.display_sample_record(preview.output.sample(1))

                                                   Seed Columns                                                    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Column Name                                ┃                                                                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ domain                                     │ Artificial Intelligence                                            │
├────────────────────────────────────────────┼────────────────────────────────────────────────────────────────────┤
│   |- topic                                 │ AI in Marketing                                                    │
├────────────────────────────────────────────┼────────────────────────────────────────────────────────────────────┤
│ sql_complexity                             │ Window Functions                                                   │
├────────────────────────────────────────────┼────────────────────────────────────────────────────────────────────┤
│ sql_task_type                              │ Window Functions                                                   │
├────────────────────────────────────────────┼────────────────────────────────────────────────────────────────────┤
│ natural_language_type                      │ an explanation of using subqueries                                 │
├────────────────────────────────────────────┼────────────────────────────────────────────────────────────────────┤
│ data_generation_mode                       │ Sequential                                                         │
└────────────────────────────────────────────┴────────────────────────────────────────────────────────────────────┘

                                                   Data Columns                                                    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Column Name                ┃ Value                                                                              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ domain_description         │ The Artificial Intelligence in focus here is specialized in the domain of writing  │
│                            │ SQL queries, particularly with an emphasis on subqueries and window functions. A   │
│                            │ key area of focus for this AI is optimizing the use of subqueries to enhance query │
│                            │ performance and readability. Subqueries are used to break down complex queries     │
│                            │ into more manageable parts, allowing for more precise data retrieval and           │
│                            │ manipulation. The AI is designed to generate and optimize subqueries in a          │
│                            │ sequential data generation mode, ensuring that each subquery contributes           │
│                            │ effectively to the overall query.                                                  │
├────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────┤
│ sql_complexity_description │ The Window Functions level describes SQL queries that utilize window functions,    │
│                            │ such as ROW_NUMBER(), RANK(), DENSE_RANK(), SUM(), AVG(), MIN(), MAX(), and other  │
│                            │ aggregate functions. These functions operate on a set of rows related to the       │
│                            │ current row within a result set partition, allowing for complex calculations and   │
│                            │ data analysis without the need for self-joins or subqueries.                       │
├────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────┤
│ sql_task_type_description  │ The `sql_task_type_description` for Window Functions is: Window Functions allow    │
│                            │ for the application of aggregate functions across a set of table rows that are     │
│                            │ related to the current row. They are commonly used in the Artificial Intelligence  │
│                            │ industry for tasks such as ranking, moving averages, and calculating cumulative    │
│                            │ values, which are essential for data analysis and feature engineering in AI        │
│                            │ models.                                                                            │
├────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────┤
│ sql_prompt                 │ Generate a SQL query to find the top 5 AI marketing campaigns based on engagement  │
│                            │ rate, including the campaign name, start date, end date, and total number of       │
│                            │ interactions. Also, include the average engagement rate per day for each campaign. │
├────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────┤
│ sql_context                │ CREATE VIEW ai_marketing_campaigns AS                                              │
│                            │ SELECT                                                                             │
│                            │     campaign_name,                                                                 │
│                            │     start_date,                                                                    │
│                            │     end_date,            

                                                    Validation                                                     
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Column Name                                                               ┃ Value                               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ sql_is_valid                                                              │ True                                │
├───────────────────────────────────────────────────────────────────────────┼─────────────────────────────────────┤
│ sql_messages                                                              │ []                                  │
└───────────────────────────────────────────────────────────────────────────┴─────────────────────────────────────┘

╭────────────────────────────────────────────────────── sql ──────────────────────────────────────────────────────╮
│ SELECT                                                                                                          │
│     campaign_name,                                                                                              │
│     start_date,                                                                                                 │
│     end_date,                                                                                                   │
│     total_interactions,                                                                                         │
│     avg_engagement_rate_per_day,                                                                                │
│     AVG(engagement_rate) OVER (PARTITION BY campaign_name) AS avg_engagement_rate                               │
│ FROM                                                                                                            │
│     ai_marketing_campaigns                                                                                      │
│ ORDER BY                                                                                                        │
│     total_interactions DESC                                                                                     │
│ LIMIT 5;                                                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### 🤔 Like what you see? Generate an entire dataset

In [27]:
# Submit a batch workflow to generate records
results = designer.submit_batch_workflow(num_records=50)

[21:00:39] [INFO] 🛜 Connecting to your Gretel Project:
[21:00:39] [INFO] 🔗 -> https://console-dev.gretel.ai/proj_2o7zwNweGeCVqAYglVkYgmCMwet
[21:00:43] [INFO] ▶️ Starting your workflow run to generate 50 records:
[21:00:43] [INFO] 🔗 -> https://console-dev.gretel.ai/workflows/w_2o7zwUp8hHds6wIMBpVOrHbjDW6/runs/wr_2o7zwjG52YmzVYb1vpN3EUENz8u


In [28]:
# Fetch the dataset
df = results.fetch_dataset(wait_for_completion=True)

[21:00:44] [INFO] 🏗️ We are still building your dataset. Workflow status: CREATED.
[21:00:44] [INFO] ⏳ Waiting for workflow run to complete...
[21:00:44] [INFO] 👀 Follow along -> https://console-dev.gretel.ai/workflows/w_2o7zwUp8hHds6wIMBpVOrHbjDW6/runs/wr_2o7zwjG52YmzVYb1vpN3EUENz8u
[21:10:55] [INFO] ✅ Fetching dataset from completed workflow run


In [29]:
# Inspect the dataset
df.head()

,sql_complexity,data_generation_mode,sql_task_type,natural_language_type,domain,topic,domain_description,sql_complexity_description,sql_task_type_description,sql_prompt,sql_context,sql,sql_explanation,sql_is_valid,sql_messages
0,Single Join,Creation,Drop Table,an explanation of using transactions,Artificial Intelligence,AI in Cybersecurity,The Artificial Intelligence in this domain foc...,The Single Join level involves a SQL query tha...,The `Drop Table` SQL task is used to remove a ...,Please provide the specific details or column ...,CREATE TABLE ai_cybersecurity_transactions (\n...,DROP TABLE ai_cybersecurity_transactions;,sql_explanation: The SQL query is dropping the...,True,[]
1,Aggregation,Augmentation,Analytics and Reporting,an explanation of creating a view,Public Services,Public Library Management,The domain description for Public Services inv...,The Aggregation level describes SQL queries th...,The `sql_task_type_description` for Analytics ...,Generate a SQL query to create a view named `l...,CREATE VIEW library_book_count_by_genre AS\nSE...,CREATE VIEW library_book_count_by_genre AS\nSE...,The SQL query creates a view named `library_bo...,True,[]
2,Multiple Join,Augmentation,Drop Table,an instruction to delete outdated records,Construction,Site Safety Standards,The domain description for the Construction su...,The `sql_complexity_description` for the Multi...,The `sql_task_type_description` for Drop Table...,Generate a SQL query to delete outdated record...,-- Create table statement for SiteSafetyInspec...,DELETE FROM SiteSafetyInspection WHERE inspect...,The SQL query deletes records from the SiteSaf...,True,[]
3,Window Functions,Creation,View Deletion,a command to filter data by date,Digital Health,Electronic Prescriptions,Digital Health encompasses the integration of ...,The Window Functions level describes SQL queri...,The `sql_task_type_description` for View Delet...,Please provide the SQL query to retrieve all e...,CREATE VIEW ElectronicPrescriptions AS\nSELECT...,DROP VIEW ElectronicPrescriptions;,The SQL query first creates a view named `Elec...,True,[]
4,Multiple Join,Editing,Delete Data,an instruction that tells a user to write an S...,Renewable Energy and Sustainability,Eco-Friendly Transportation,The domain description for Renewable Energy an...,The `sql_complexity_description` for the Multi...,The `sql_task_type_description` for Delete Dat...,Please write an SQL query to find the total nu...,"SELECT city, COUNT(*) AS total_charging_statio...",DELETE FROM electric_vehicle_charging_stations...,The SQL query deletes all records from the `el...,True,[]
